# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,381.679066,-0.140226,1234,381.538840,381.819293,763.077680,763.638585
6,903.159220,-0.396483,1234,902.762736,903.555703,1805.525473,1807.111406
7,788.976874,-0.972013,1234,788.004861,789.948887,1576.009722,1579.897774
8,294.362133,-0.219654,1234,294.142479,294.581787,588.284958,589.163574
9,486.688733,-0.468853,1234,486.219879,487.157586,972.439758,974.315172
10,993.530112,0.870491,1234,994.400603,992.659622,1988.801206,1985.319244
11,877.678922,0.331111,1234,878.010033,877.347810,1756.020066,1754.695621
12,596.118512,-0.150865,1234,595.967648,596.269377,1191.935295,1192.538754
13,883.521333,0.357702,1234,883.879035,883.163631,1767.758071,1766.327263
14,134.729210,0.404850,1234,135.134060,134.324360,270.268121,268.648721


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-381.538840,381.819293
6,-902.762736,903.555703
7,-788.004861,789.948887
8,-294.142479,294.581787
9,-486.219879,487.157586
10,-994.400603,992.659622
11,-878.010033,877.347810
12,-595.967648,596.269377
13,-883.879035,883.163631
14,-135.134060,134.324360


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-381.538840,381.819293
6,-902.762736,903.555703
7,-788.004861,789.948887
8,-294.142479,294.581787
9,-486.219879,487.157586
10,-994.400603,992.659622
11,-878.010033,877.347810
12,-595.967648,596.269377
13,-883.879035,883.163631
14,-135.134060,134.324360


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-381.53883992,  381.81929273],
       [-902.76273629,  903.5557029 ],
       [-788.00486092,  789.94888677],
       [-294.14247876,  294.58178676],
       [-486.21987909,  487.15758604],
       [-994.40060303,  992.65962192],
       [-878.0100328 ,  877.34781039],
       [-595.96764754,  596.26937696],
       [-883.8790353 ,  883.16363143],
       [-135.13406028,  134.32436033]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,761.077680,-763.638585
6,1803.525473,-1807.111406
7,1574.009722,-1579.897774
8,586.284958,-589.163574
9,970.439758,-974.315172
10,1986.801206,-1985.319244
11,1754.020066,-1754.695621
12,1189.935295,-1192.538754
13,1765.758071,-1766.327263
14,268.268121,-268.648721


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.381679,0.429887
6,0.903159,0.301758
7,0.788977,0.013994
8,0.294362,0.390173
9,0.486689,0.265573
10,0.993530,0.935245
11,0.877679,0.665556
12,0.596119,0.424568
13,0.883521,0.678851
14,0.134729,0.702425


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,381.679066,-0.140226
6,903.159220,-0.396483
7,788.976874,-0.972013
8,294.362133,-0.219654
9,486.688733,-0.468853
10,993.530112,0.870491
11,877.678922,0.331111
12,596.118512,-0.150865
13,883.521333,0.357702
14,134.729210,0.404850


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
